Install all the dependencies

In [ ]:
!pip install langchain google-cloud-aiplatform mlflow yfinance exa_py brave-search


Import all the dependencies required

In [ ]:
import os
import mlflow
import yfinance as yf
from langchain.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_google_vertexai import VertexAI
from langchain_community.tools import BraveSearch

In [ ]:
# VertexAI setup
llm = VertexAI(model="gemini-2.0-flash", temperature=0.2)

In [ ]:
#MLFlow setup
mlflow.set_tracking_uri("http://20.75.92.162:5000/")
mlflow.set_experiment("market_sentiment_analyzer")
mlflow.start_run(run_name="sentiment_pipeline")

In [ ]:
# Update the company as needed, for ex Google
company_name = "Google"
mlflow.log_param("company_name", company_name)

In [ ]:
# =========================================================
# Cell 7: Step 2 - Stock Code Lookup
# Reference: code4_function_calling_with_gemini.ipynb
# =========================================================

def get_stock_code(company_name: str) -> str:
    try:
        ticker = yf.Ticker(company_name)
        return ticker.ticker if hasattr(ticker, "ticker") else "N/A"
    except Exception:
        return "N/A"

stock_code = get_stock_code(company_name)
mlflow.log_param("stock_code", stock_code)

print(f"Stock Code for {company_name}: {stock_code}")


In [ ]:
# =========================================================
# Cell 8: Step 3 - Fetch News
# Reference: code4_function_calling_with_gemini.ipynb & code5_Langchain_and_Langgraph_Chains.ipynb
# =========================================================

news_tool = BraveSearch()
results = news_tool.run(company_name)

news_summaries = [r["title"] + " - " + r["snippet"] for r in results[:5]]
mlflow.log_param("news_count", len(news_summaries))

print("Top 5 News Headlines:")
for i, news in enumerate(news_summaries, 1):
    print(f"{i}. {news}")


In [ ]:
# =========================================================
# Cell 9: Step 4 - Define Output JSON Schema
# Reference: code7_NLP2SQL_Chains.ipynb
# =========================================================

response_schemas = [
    ResponseSchema(name="company_name", description="Name of the company"),
    ResponseSchema(name="stock_code", description="Stock ticker symbol"),
    ResponseSchema(name="newsdesc", description="Brief description of news"),
    ResponseSchema(name="sentiment", description="Positive/Negative/Neutral"),
    ResponseSchema(name="people_names", description="List of people mentioned"),
    ResponseSchema(name="places_names", description="List of places mentioned"),
    ResponseSchema(name="other_companies_referred", description="Other companies mentioned"),
    ResponseSchema(name="related_industries", description="Industries referenced"),
    ResponseSchema(name="market_implications", description="Implications for the market"),
    ResponseSchema(name="confidence_score", description="Float between 0 and 1"),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


In [ ]:
# =========================================================
# Cell 10: Step 4 - Prompt Template for Gemini
# Reference: code2_prompt_version_control_management.ipynb
# =========================================================

template = """
Analyze the following recent news about {company_name} ({stock_code}) 
and generate a structured sentiment profile.

News:
{news_summaries}

{format_instructions}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["company_name", "stock_code", "news_summaries"],
    partial_variables={"format_instructions": format_instructions}
)


In [ ]:
# =========================================================
# Cell 12: Step 5 - Close mlflow Run
# =========================================================

mlflow.end_run()
